In [1]:
import pywt
import matplotlib.pyplot as plt
import numpy as np
import os
import glob

# Step 1: Define paths for input and output datasets
root_input_folder = 'Dataset/Extended_Data_1'
root_output_folder = 'wavelet_results_1'
os.makedirs(root_output_folder, exist_ok=True)  # Create root output folder if it doesn't exist

# Step 2: Normalize array using Min-Max normalization
def min_max_normalize(arr):
    """Normalize the array between 0 and 1."""
    min_val = np.min(arr)
    max_val = np.max(arr)
    if max_val - min_val == 0:  # Handle case where all values are the same
        return np.zeros_like(arr)  # Return array of zeros
    return (arr - min_val) / (max_val - min_val)

# Step 3: Loop through both A_extended and E_extended folders
for folder in ['A_extended', 'E_extended']:
    input_folder = os.path.join(root_input_folder, folder)
    output_folder = os.path.join(root_output_folder, folder)
    os.makedirs(output_folder, exist_ok=True)  # Create output folder if it doesn't exist

    # Step 4: Get a list of all .txt files in the current input folder
    txt_files = glob.glob(os.path.join(input_folder, '*.txt'))

    # Step 5: Process each .txt file
    for txt_file in txt_files:
        # Extract filename without extension to name the output folder
        filename = os.path.splitext(os.path.basename(txt_file))[0]
        file_output_folder = os.path.join(output_folder, filename)
        os.makedirs(file_output_folder, exist_ok=True)  # Create folder for each file's results

        # Step 6: Load EEG data from the current .txt file
        eeg_data = np.loadtxt(txt_file)

        # Step 7: Perform Discrete Wavelet Transform (DWT) using 'db4'
        wavelet = 'db4'
        coeffs = pywt.wavedec(eeg_data, wavelet, level=5)

        # Step 8: Reconstruct D1, D2, D3, D4, D5, and A5 from the coefficients
        A5 = pywt.waverec([coeffs[0]] + [np.zeros_like(c) for c in coeffs[1:]], wavelet)
        D5 = pywt.waverec([np.zeros_like(coeffs[0])] + [coeffs[1]] + [np.zeros_like(c) for c in coeffs[2:]], wavelet)
        D4 = pywt.waverec([np.zeros_like(coeffs[0])] + [np.zeros_like(coeffs[1])] + [coeffs[2]] + [np.zeros_like(c) for c in coeffs[3:]], wavelet)
        D3 = pywt.waverec([np.zeros_like(coeffs[0])] + [np.zeros_like(coeffs[1])] + [np.zeros_like(coeffs[2])] + [coeffs[3]] + [np.zeros_like(c) for c in coeffs[4:]], wavelet)
        D2 = pywt.waverec([np.zeros_like(coeffs[0])] + [np.zeros_like(coeffs[1])] + [np.zeros_like(coeffs[2])] + [np.zeros_like(coeffs[3])] + [coeffs[4]] + [np.zeros_like(coeffs[5])], wavelet)
        D1 = pywt.waverec([np.zeros_like(coeffs[0])] + [np.zeros_like(coeffs[1])] + [np.zeros_like(coeffs[2])] + [np.zeros_like(coeffs[3])] + [np.zeros_like(coeffs[4])] + [coeffs[5]], wavelet)

        # Step 9: Save the reconstructed components as .txt files
        components = {'A5': A5, 'D5': D5, 'D4': D4, 'D3': D3, 'D2': D2, 'D1': D1}
        for comp_name, comp_data in components.items():
            np.savetxt(os.path.join(file_output_folder, f'{comp_name}.txt'), comp_data)

        # Step 10: Create a single figure with subplots for all components
        fig, axs = plt.subplots(6, 1, figsize=(10, 15))  # 6 rows, 1 column

        # Plot each component in a separate subplot
        for i, (comp_name, comp_data) in enumerate(components.items()):
            axs[i].plot(comp_data)
            axs[i].set_title(f'{comp_name} - {filename}')
            axs[i].set_xlabel('Samples')
            axs[i].set_ylabel('Amplitude')
            axs[i].grid(True)

        plt.tight_layout()  # Adjust layout to prevent overlap

        # Step 11: Save the consolidated figure
        plt.savefig(os.path.join(file_output_folder, f'{filename}_wavelet_components.png'))
        plt.close(fig)
